<p align="center">
  <img src="https://huggingface.co/speakleash/Bielik-7B-Instruct-v0.1/raw/main/speakleash_cyfronet.png">
</p>

%%capture
!pip install -q streamlit accelerate transformers bitsandbytes
!npm install -g localtunnel

## Prosta aplikacja z wykorzystaniem Streamlit i Bielika w wersji 4-bit

In [ ]:
% % writefile
app.py
from threading import Thread

import streamlit as st
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextIteratorStreamer

MODEL_NAME = 'speakleash/Bielik-7B-Instruct-v0.1'
LOAD_4_BIT = True


@st.cache_resource  # Dekorator, żeby model i inne rzeczy były trzymane w pamięci cache
def prepare_model() -> tuple:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    quantization_config = None
    if LOAD_4_BIT:
        quantization_config = BitsAndBytesConfig(load_in_4bit=LOAD_4_BIT, bnb_4bit_compute_dtype=torch.bfloat16)

    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.bfloat16,
                                                 quantization_config=quantization_config)
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    return tokenizer, model, streamer


def generate_text(tokenizer: AutoTokenizer, model: AutoModelForCausalLM,
                  streamer: TextIteratorStreamer,
                  prompt: str = "", temperature: float = 0.4,
                  top_k: int = 200, top_p: float = 0.95, max_tokens: int = 512) -> str:
    # Przygotowanie promptu przez tokenizer
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Przygotowanie zmiennych dla modelu
    generation_kwargs = {
            "input_ids": inputs["input_ids"],
            "max_new_tokens": max_tokens,
            "streamer": streamer,
            "do_sample": True if temperature else False,
            'temperature': temperature,
            'top_k': top_k,
            'top_p': top_p
    }

    # Uruchamiamy generowanie modelu w osobnym wątku
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    for output in streamer:
        yield output  # Zwracamy fragmenty odpowiedzi


### ------------------------------------------------------------------------ ###
# Aplikacja Streamlit

# Historia czatu
if "messages" not in st.session_state:
    st.session_state.messages = []

st.title("Bielik-v0.1-7B (4-bit) - LLM App")

# Ładowanie modelu
with st.status("Pobieranie i przygotowanie modelu...", expanded=True) as status:
    tokenizer, model, streamer = prepare_model()
    status.update(label="Model gotowy do rozmowy!", state="complete", expanded=False)

# Ustawienia parametrów modelu
col_temp, col_maxtokens, col_topk, col_topp = st.columns([1, 1, 1, 1])
with col_temp:
    temperature = st.slider("Temperatura", 0.01, 2.0, 0.4)
with col_maxtokens:
    max_tokens = st.slider("Max liczba tokenów:", 1, 4096, 512)
with col_topk:
    top_k = st.slider("Top_k", 1, 500, 200)
with col_topp:
    top_p = st.slider("Top_p", 0.01, 1.0, 0.95)

# Historia konwersacji
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Czat z modelem
if human_prompt := st.chat_input("Witaj, o czym dzisiaj porozmawiamy?"):
    # Dodanie specjalnych tokenów, żeby model działał poprawnie
    prompt = f"<s>[INST]{human_prompt}[/INST]"

    # Zapisanie promptu do historii
    st.session_state.messages.append({"role": "user", "content": human_prompt})

    # Pokazanie wiadomości od użytkownika
    with st.chat_message("user"):
        st.markdown(human_prompt)

    # Odpowiedź modelu
    with st.chat_message("assistant"):
        # Streaming - czyli wiadomość będzie uzupełniana jak tylko model zwróci kolejny token
        # czyli będą się pojawiać kolejne tokeny i nie będziemy czekać na całą wiadomość
        response_stream = st.write_stream(generate_text(tokenizer,
                                                        model,
                                                        streamer,
                                                        prompt,
                                                        temperature,
                                                        top_k,
                                                        top_p,
                                                        max_tokens))

    # Zapisanie odpowiedzi modelu do historii czatu
    st.session_state.messages.append({"role": "assistant", "content": response_stream})

In [ ]:
# Po uruchomieniu będzie adres IP - należy go skopiować i przejść na stronę, która będzie poniżej
!streamlit run app.py &>/content/logs.txt &
!npx localtunnel --port 8501 & curl ipv4.icanhazip.com